In [1]:
from search.Problem import Problem
from search.Search import aStarSearch, breadthFirstSearch, ipa_star

# L'algorithme A*
A* est un algorithme de recherche permettant de trouver le chemin dans un graphe entre un noeud de départ et un noeud d'arrivé.
Cette algorithme repose sur deux éléments importants:
 - Un problème formalisé
 - Une heuristique admissible et consistante
 
L'algorithme consiste à explorer les noeuds du graphes en passant par les meilleurs noeuds, que l'on classe selon leur valeur $f(n) = g(n) + h(n)$ où $g(n)$ est la profondeur du noeud $n$ et $h(n)$ l'heuristique au noeud $n$.

## Formalisation du problème
Il est nécessaire de réfléchir à la formalisation du problème que l'on souhaite résoudre.

### La tour de Hanoï
Prenons le jeu de la tour de Hanoï et essayons de le modéliser afin de la résoudre à l'aide de l'algorithme A*.
Le principe est simple, on dispose de trois emplacements qui peuvent accueillir des disques de tailles différentes. Il faut déplacer la tour de gauche sur l'emplacement de droite avec le moins de coups possibles. Chaque disque ne peut être posé que sur un disque de diamètre supérieur au sien.

In [2]:
import copy

class Hanoi(Problem):
    def __init__(self):
        start = [[i for i in reversed(range(5))], [], []]
        end = [[], [], [i for i in reversed(range(5))]]
        super().__init__(start, end)
        
    def _buildNext(self, prev):
        children = []
        
        for i in range(3):
            board = copy.deepcopy(prev)

            if len(board[i]) > 0:
                piece = board[i].pop()
                
                possible_moves = [0, 1, 2]
                possible_moves.remove(i)
                
                for j in possible_moves:
                    if len(board[j]) == 0 or board[j][-1] > piece:
                        next_board = copy.deepcopy(board)
                        next_board[j].append(piece)
                        children.append(next_board)
                
        return children    

## Heuristique
Une heuristique est une fonction qui, pour un noeud donné, donne le coût permettant d'atteindre le noeud but.

On dit d'une heuristique qu'elle est **admissible** lorsqu'elle est inférieure ou égale au coût réel pour se rendre au but.
On dit également d'une heuristique qu'elle est **consistante** (ou monotone) lorsque, pour chaque nœud $n$ et chaque successeur $n’$ de $n$ produit par n’importe quelle action, le coût estimé pour atteindre le but à partir de $n$ n’est pas supérieur au coût de l’étape pour aller à $n’$ plus le coût estimé pour atteindre le but à partir de $n’$.

Tout l'art des algorithmes de recherche réside dans le développement d'heuristiques admissibles et consistantes afin de préserver l'optimalité de l'algorithme.

L'utilisation de certaines heuristiques dites triviales équivaut à l'utilisation d'algorithmes de recherches dérivés:
- Si $f(n) = g(n)$: algorithme du coût uniforme, qui équivaut à une exploration en largeur d'abord lorsque les coûts pour passer d'un noeud parent à un noeud fils sont égaux (Breadth First Search)
- Si $h(n) = 0 \Rightarrow f(n) = g(n)$: algorithme glouton (Best first search)

Créons une heuristique pour notre problème. Commençons simplement par poser $h(n) = 0$, de cette façon, l'algorithme agira comme un algorithme glouton et va tester toutes les possibilités en parcourant le graphe des coups en largeur. On trouvera ainsi assurément la solution la plus courte (en nombre de coups) mais au détriment du temps de recherche.

In [3]:
def heuristic(state, goal):
    return 0

## Résolution
Voyons le résultat de notre algorithme

In [4]:
problem = Hanoi()
result = aStarSearch(problem, heuristic)

print(result)

Solution found in 0.051723000000000074 seconds
Length of the solution: 31
Nodes opened: 233


On voit donc que l'algorithme a trouvé le chemin le plus court (le jeu de la tour de Hanoï est bien connu, il est admis que la tour de Hanoïe à 5 disques ne peut se résoudre en moins de 31 mouvements).

En cherchant la solution, l'algorithme a parcouru 233 noeuds du graphes des coups jouables.

## Une autre heuristique ?
Cherchons maintenant une heuristique un peu plus intelligente pour estimer la distance entre un noeud et le noeud que l'on cherche à atteindre.

On peut admettre que le nombre de coup qu'il reste à jouer est inférieur au nombre de pièces sur la position 3

Posons donc $h(n) = 5 - \textit{nb_pieces_en_3}$

Ceci est une heuristique admissible, en effet, lorsqu'il ne reste plus qu'une seule pièce à déplacer l'heuristique proposée est égale au coût réel de déplacement. Lorsqu'il reste plus d'une pièce à déplacer, l'heuristique proposée est toujours inférieure au nombre de déplacement réel du fait de la mécanisme de déplacement ; un disque étant toujours posé sur un disque de diamètre plus grand, pour déplacer $n$ pièces, il faudra toujours plus de $n$ mouvements.

In [5]:
def wrong_place_heuristic(state, goal):
    return 5 - len(state.Value[2])

In [6]:
result2 = aStarSearch(problem, wrong_place_heuristic)
print(result2)

Solution found in 0.03436000000000006 seconds
Length of the solution: 31
Nodes opened: 178


On s'aperçoit qu'avec une heuristique plus "intelligente", l'algorithme présente de bien meilleur performance, il a parcouru 76% de noeuds en moins pour trouver la solution optimale en moins de temps.

## Encore un problème
Le jeu de la tour de Hanoï est un problème extrêmement simple et peut même être résolu rapidement par des algorithmes récursifs, mais il permet de tester l'efficacité des algorithmes de recherche.

Testons maintenant notre algorithme sur un problème régulièrement utilisé dans le domaine des jeux vidéos : la recherche de chemin dans un labyrinthe.

In [42]:
class Maze(Problem):
    def __init__(self, maze):
        start = (4, 0)
        end = (1, 9)
        self.maze = maze
        super().__init__(start, end)
        
    def _buildNext(self, prev):
        children = []
        x, y = prev
        if x > 0 and self.maze[x-1][y] != '#':
            children.append((x-1, y))
        if x < len(self.maze)-1 and self.maze[x+1][y] != '#':
            children.append((x+1, y))
        if y > 0 and self.maze[x][y-1] != '#':
            children.append((x, y-1))
        if y < len(self.maze[0])-1 and self.maze[x][y+1] != '#':
            children.append((x, y+1))
        return children

Dans ce cas, l'heuristique la plus simple reste la distance entre les deux points. On peut utiliser la distance euclidienne, ou la distance de manhattan qui donnerait la distance exacte s'il n'y avait pas les obstacles.

In [43]:
def euclidian_heuristic(state, goal):
    x1, y1 = state.Value
    x2, y2 = goal.Value
    return ((x1 - x2) ** 2 + (y1 - y2) ** 2)**0.5

In [44]:
def manhattan_heuristic(state, goal):
    x1, y1 = state.Value
    x2, y2 = goal.Value
    return abs(x1 - x2) + abs(y1 - y2)

In [45]:
my_maze = [
    '  ####    ',
    '     #   A',
    '  #  # # #',
    ' ###   # #',
    'D  ##  ###',
    '    ##    ',
]

maze_problem = Maze(my_maze)

In [48]:
result3 = aStarSearch(maze_problem, euclidian_heuristic)
print(result3)

Solution found in 0.001072000000000184 seconds
Length of the solution: 16
Nodes opened: 28


In [49]:
result4 = aStarSearch(maze_problem, manhattan_heuristic)
print(result4)

Solution found in 0.0009369999999999656 seconds
Length of the solution: 16
Nodes opened: 27
